In [1]:
access_token = 'EAAH24PQU4egBADGV6Xd8EnwaCmEjvlKZCLYsymsSZBVCPBvvZBtcOnMuotunZAGh4BBvAP9dHYLAnS5J70UOIXDgEbxhgHbGXOTWGAESBfSlJGj6GDZAlDvh5KYzAztgsaOxbGSlVLaT29P00gmsSLBNGXiSZBT71mRUa5rsFo2I8ZC6d4I0oBFFZCaqbyKbbCMZD'
host = '78.46.226.67'
port = '5432'
database ='postgres'
user='kirill_k'
password='swfr35$#'

address = 'postgresql://' + user +':' + password + '@' + host +':' + port + '/' + database

In [2]:
!pip install facebook-business
import psycopg2

from tqdm import tqdm

import pandas as pd
import numpy as np

from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.campaign import Campaign
from facebook_business.exceptions import FacebookRequestError, FacebookBadObjectError
from facebook_business.adobjects.user import User
from facebook_business.api import FacebookAdsApi
from psycopg2 import OperationalError

import json

import datetime
from datetime import timedelta, datetime, date, time
import time

     |████████████████████████████████| 1.0MB 7.0MB/s 
     |████████████████████████████████| 10.1MB 31.2MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=11471001023e25cae28bcb77f7a4d85b11c3d31e990270cf76cbdb2445d4c6a6
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
  Created wheel for curlify: filename=curlify-2.2.1-cp36-none-any.whl size=2660 sha256=620d09ca2248d5686a77cacd300dacbffd510e99b8994195e7df716ab1f91706
  Stored in directory: /root/.cache/pip/wheels/b7/5e/e5/6f90a2b21faf7b71748fcba58920ad77850c25b8f70927d418
Successfully built pycountry curlify


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
def connection_to_database(host, port, database, user, password):
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    cur = connection.cursor()
    return cur, connection

def reconnection_to_database(host, port, database, user, password, connection):
    if connection.closed > 0:
        connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
        cur = connection.cursor()
    else:
        try:
            connection.cursor().execute('SELECT 1')
        except OperationalError:
            connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
            cur = connection.cursor()
    return connection.cursor(), connection

def store_object(object_data, table):
    fields = ','.join(object_data.keys())
    values = ','.join(map(lambda x:f"%({x})s",object_data.keys()))
    query =   f"INSERT INTO {table} ({fields}) values ({values})"
    cur.execute(query, object_data)
    connection.commit()
    
def delete_object(object_id, table):
    query_to_delete = f"""delete  from {table} where id = %s"""  
    cur.execute(query_to_delete, vars=(object_id, ))
    
def store_deleted_object(object_id, object_type, account_id, table):
    today = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    query =  f"INSERT INTO {table} values (%s, %s, %s, %s)"
    cur.execute(query, vars=(object_id, account_id, object_type,today,))
    connection.commit()

def dict2json(dictionary):
    return json.dumps(dictionary, ensure_ascii=False, sort_keys=True)

def modify_object(object_data):
    for key in object_data.keys():
        if type(object_data[key]) in (list, dict):
            object_data[key] = dict2json(object_data[key])
    object_data['recording_date'] = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    
    return object_data

In [4]:
print('start processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

FacebookAdsApi.init(access_token=access_token)

try:
    cur.close()
    cur, connection = connection_to_database(host, port, database, user, password)
except NameError:
    cur, connection = connection_to_database(host, port, database, user, password)

me = User(fbid='me')
accounts = list(me.get_ad_accounts())
accounts = list(map(lambda x:x.export_all_data(), accounts))
account_ids = list(map(lambda x:x['account_id'], accounts))

object_type = 'campaign'
table = 'property_' + object_type + 's'

cur.execute(f""" CREATE TABLE if not exists {table}(
    id VARCHAR(50) unique primary key,    account_id VARCHAR(50),     name VARCHAR(200),  source_campaign JSONB,
    source_campaign_id VARCHAR(50), status VARCHAR(30),     updated_time timestamp,     start_time timestamp, 
    stop_time timestamp, spend_cap VARCHAR(30), adlabels JSONB, boosted_object_id VARCHAR(50),
    promoted_object JSONB, last_budget_toggling_time timestamp, special_ad_category VARCHAR(30),
    bid_strategy VARCHAR(50),  configured_status VARCHAR(30),     created_time timestamp, 
    effective_status VARCHAR(30), daily_budget VARCHAR(30), objective  VARCHAR(30), budget_rebalance_flag BOOLEAN, 
    buying_type VARCHAR(30), pacing_type JSONB,  can_create_brand_lift_study BOOLEAN, 
    can_use_spend_cap BOOLEAN, topline_id VARCHAR(30), budget_remaining VARCHAR(30), lifetime_budget VARCHAR(30),
    recording_date timestamp without time zone)""")
connection.commit()

cur.execute(f""" CREATE TABLE if not exists deleted_objects(
    object_id VARCHAR(50) unique primary key,    account_id VARCHAR(50),     
    object_type VARCHAR(30),    recording_date timestamp without time zone)""")
connection.commit()

fields = list(Campaign.Field.__dict__.keys())
exclude_list = ['__module__', '__doc__', 'adbatch' , 'execution_options', 'iterative_split_test_configs',
                'upstream_events']

fields = list(set(fields)-set(exclude_list ))

query = """SELECT DISTINCT t.*
FROM (SELECT DISTINCT actions.object_id as campaign_id, actions.account_id, 
property_campaigns.recording_date is NULL as has_not_data
FROM actions LEFT JOIN property_campaigns on actions.object_id = property_campaigns.id
WHERE object_type = 'CAMPAIGN_GROUP' and 
(property_campaigns.recording_date is NULL or actions.event_time > property_campaigns.recording_date)
UNION
SELECT DISTINCT insights.campaign_id, insights.account_id, False as has_not_data
from insights
left join property_campaigns on insights.campaign_id = property_campaigns.id
where property_campaigns.id is null) t
LEFT JOIN deleted_objects on  t.campaign_id = deleted_objects.object_id
WHERE deleted_objects.object_id IS NULL"""

campaign_list = pd.read_sql(query, connection)
data = pd.read_sql(query, connection)
data = data[data['account_id'].isin(account_ids)]
campaigns = dict(zip(list(data['campaign_id']), data['has_not_data']))
accounts =  dict(zip(list(data['campaign_id']), list(data['account_id'])))

print('start uploading \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))


for campaign_id in tqdm(campaigns.keys()):
    try:
        campaign_data = Campaign(campaign_id).api_get(fields = fields).export_all_data()
        campaign_data = modify_object(campaign_data)   
        cur, connection = reconnection_to_database(host, port, database, user, password, connection)
        if campaigns[campaign_id]:
            store_object(campaign_data, table)
        else:
            delete_object(campaign_id, table)
            store_object(campaign_data, table)
        connection.commit()
        
    except FacebookRequestError as error:
        code = error._body['error']['code']
        subcode = error._body['error']['error_subcode']
        if code == 100 and (subcode == 33 or subcode == 1487221):
            cur, connection = reconnection_to_database(host, port, database, user, password, connection)
            store_deleted_object(campaign_id, object_type, accounts[campaign_id], 'deleted_objects')
#        print('An unknown error occurred for adset ' + adset_id)

print('start refreshing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

# cur, connection = reconnection_to_database(host, port, database, user, password, connection)
# query = """REFRESH MATERIALIZED VIEW mview_adset_atributes"""
# cur.execute(query)
# connection.commit()

print('end processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

start processing 	2020-07-13 19:36:44


  0%|          | 0/40 [00:00<?, ?it/s]

start uploading 	2020-07-13 19:36:45


/usr/local/lib/python3.6/dist-packages/facebook_business/utils/api_utils.py:30: UserWarning: Value of bid_strategy is not be compatible. Expect BidStrategy; got <class 'str'>
  warnings.warn(message)
100%|██████████| 40/40 [00:20<00:00,  1.91it/s]

start refreshing 	2020-07-13 19:37:06
end processing 	2020-07-13 19:37:06
